In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'GeForce GTX 1060 3GB'

In [99]:
torch.cuda.memory_allocated()

125440

In [100]:
torch.cuda.memory_cached()

2097152

In [7]:
a = torch.FloatTensor([1.0, 2.0])

In [8]:
a

tensor([1., 2.])

In [9]:
a.device

device(type='cpu')

In [11]:
a = torch.FloatTensor([1.0, 2.0]).cuda()

In [12]:
a.device

device(type='cuda', index=0)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [49]:
torch.manual_seed(32)
model = Model()

In [50]:
next(model.parameters()).is_cuda

False

In [51]:
gpumodel = model.cuda()

In [52]:
next(gpumodel.parameters()).is_cuda

True

In [87]:
torch.manual_seed(32)
gpumodel = Model().cuda()
next(gpumodel.parameters()).is_cuda

True

In [88]:
df = pd.read_csv('../Data/iris.csv')

In [89]:
X = df.drop("target", axis=1).values

In [90]:
y = df["target"].values

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [92]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()

In [93]:
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [94]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [95]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(),lr=0.01)

In [96]:
import time
start_time = time.time()

epochs = 100
losses = []

for i in range(epochs):
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    if i%10 == 0:
        print(f"Epoch: {i} loss: {loss.item()}")
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
end = time.time() - start_time
print(f"TOTAL Time: {end}")

Epoch: 0 loss: 1.1507114171981812
Epoch: 10 loss: 0.9377313256263733
Epoch: 20 loss: 0.779825747013092
Epoch: 30 loss: 0.6099399924278259
Epoch: 40 loss: 0.40079933404922485
Epoch: 50 loss: 0.25436320900917053
Epoch: 60 loss: 0.15053053200244904
Epoch: 70 loss: 0.10086945444345474
Epoch: 80 loss: 0.08128313720226288
Epoch: 90 loss: 0.0723142921924591
TOTAL Time: 0.2781953811645508


In [98]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f"{i+1:2}. {str(y_val):38} {y_test[i]}")
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f"\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct")

 1. tensor([-2.1252,  4.8064, -0.8628], device='cuda:0') 1
 2. tensor([-1.7985,  5.3098, -1.5449], device='cuda:0') 1
 3. tensor([  6.3542,   0.8438, -10.0541], device='cuda:0') 0
 4. tensor([-3.9123,  4.5958,  1.1693], device='cuda:0') 1
 5. tensor([-7.4713,  3.2021,  5.7853], device='cuda:0') 2
 6. tensor([-10.4976,   1.6460,   9.6297], device='cuda:0') 2
 7. tensor([  6.3201,   0.9917, -10.1532], device='cuda:0') 0
 8. tensor([  7.0468,   0.7059, -10.9137], device='cuda:0') 0
 9. tensor([-7.2061,  3.3477,  5.3565], device='cuda:0') 2
10. tensor([-9.3960,  2.5760,  8.1033], device='cuda:0') 2
11. tensor([-9.8807,  2.3475,  8.7141], device='cuda:0') 2
12. tensor([ 6.2748,  0.6655, -9.7613], device='cuda:0') 0
13. tensor([-9.3142,  2.1880,  8.1947], device='cuda:0') 2
14. tensor([-3.7803,  4.5050,  1.0752], device='cuda:0') 1
15. tensor([-7.8657,  3.0118,  6.2304], device='cuda:0') 2
16. tensor([-1.8867,  5.1572, -1.3345], device='cuda:0') 1
17. tensor([-5.7006,  3.5030,  3.6696], devi